In [5]:
def define_searchspace(trial):
    model_type = trial.suggest_categorical('model', ['fsr_model.LSTM', 'fsr_model.CNN_LSTM', 'fsr_model.ANN'])
    if model_type == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model_type == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model_type == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    trial.suggest_categorical('scaler', [
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])

In [6]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
import datasource
from trainable import Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        ray.tune.with_parameters(Trainable, data=datasource.get_data()),
        {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=-1,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ),
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ),
) 
results = tuner.fit()

[I 2023-07-02 04:23:04,111] A new study created in memory with name: optuna


2023-07-02 04:23:04,129	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
Trainable_1148c251,2023-07-02_04-25-02,True,DESKTOP-0P789CI,2,27039.2,1.68783e+18,172.26.215.93,162320,80658.1,8.84129,4.01877,8.84129,1688239502,2,1148c251
Trainable_792c775a,2023-07-02_04-25-09,True,DESKTOP-0P789CI,1,147143,1.05496e+20,172.26.215.93,162528,294226,0.800435,0.800435,0.800435,1688239509,1,792c775a
Trainable_79a0a71b,2023-07-02_04-24-40,True,DESKTOP-0P789CI,100,26607.5,66.8692,172.26.215.93,161906,63913.7,90.4757,0.863655,90.4757,1688239480,100,79a0a71b
Trainable_8f8decb8,2023-07-02_04-24-46,True,DESKTOP-0P789CI,1,238298,1.3376e+17,172.26.215.93,162153,595574,2.30414,2.30414,2.30414,1688239486,1,8f8decb8
Trainable_bc0d4e96,2023-07-02_04-25-17,False,DESKTOP-0P789CI,1,25350.6,30.8426,172.26.215.93,162685,62214,2.18227,2.18227,2.18227,1688239517,1,bc0d4e96


2023-07-02 04:23:10,309	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 1.339 s, which may be a performance bottleneck.
2023-07-02 04:23:10,312	WARNING util.py:315 -- The `process_trial_result` operation took 1.342 s, which may be a performance bottleneck.
2023-07-02 04:23:10,314	WARNING util.py:315 -- Processing trial results took 1.344 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-07-02 04:23:10,316	WARNING util.py:315 -- The `process_trial_result` operation took 1.346 s, which may be a performance bottleneck.
(_WandbLoggingActor pid=161957) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=161957) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=161957) wandb: Run data is saved locally in /home/seokj/ray_results/Trainable_2023-07-02_04-23-04/Trainable_79a0a71b_1_criterion=torch_nn_MSELoss,model=fsr_model_LSTM,hidden_siz